# 2. Update the existing code

## Group Members

### - Cho Yunah: 2017010255 

### - Lee Sihyeon: 2017010146 

# Classification using Haberman's Survival Data Set

This is a reimplementation of the K-Nearest Neighbors algorithm using plain Python.

In my opinion it is important to understand the "low level", not just the abstraction.

Data Set: Haberman's Survival Data Set

In [1]:
import math

In [2]:
data = []

##### Import the data and append it to the list

 [age_of_the_patient, year_of_operation, number_of_nodes_detected, survival_status]
 
 
 Check the data set's link above for more details

In [3]:
with open('haberman.data', 'r') as f:
    for line in f.readlines():
        atributes = line.strip('\n').split(',')
        data.append([int(x) for x in atributes])

##### Auxiliary function to help the visualization

Also returns key information of the data set

In [4]:
def info_dataset(data, verbose=True):
    label1, label2 = 0, 0
    data_size = len(data)
    for datum in data:
        if datum[-1] == 1:
            label1 += 1
        else:
            label2 += 1
    if verbose:
        print('Total of samples: %d' % data_size)
        print('Total label 1: %d' % label1)
        print('Total label 2: %d' % label2)
    return [len(data), label1, label2]

In [5]:
info_dataset(data)

Total of samples: 306
Total label 1: 225
Total label 2: 81


[306, 225, 81]

##### Standarization

In [6]:
type(data)

list

In [7]:
import pandas as pd

In [8]:
pd_data = pd.DataFrame(data=data, columns=['Age','Operation','Nodes','Status'])

In [9]:
pd_data

,Age,Operation,Nodes,Status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1
...,...,...,...,...
301,75,62,1,1
302,76,67,0,1
303,77,65,3,1
304,78,65,1,2


In [10]:
pd_data.mean()

Age          52.457516
Operation    62.852941
Nodes         4.026144
Status        1.264706
dtype: float64

In [11]:
pd_data.std()

Age          10.803452
Operation     3.249405
Nodes         7.189654
Status        0.441899
dtype: float64

In [12]:
stand_data = (pd_data - pd_data.mean()) / pd_data.std()

In [13]:
stand_data['Status'] = pd_data['Status']

In [14]:
stand_data

,Age,Operation,Nodes,Status
0,-2.078735,0.353006,-0.420903,1
1,-2.078735,-0.262492,-0.142725,1
2,-2.078735,0.660755,-0.559991,1
3,-1.986172,-1.185738,-0.281814,1
4,-1.986172,0.660755,-0.003636,1
...,...,...,...,...
301,2.086600,-0.262492,-0.420903,1
302,2.179163,1.276252,-0.559991,1
303,2.271726,0.660755,-0.142725,1
304,2.364289,0.660755,-0.420903,2


In [15]:
stand_data_list = stand_data.values.tolist()

##### Define the train/total percentage

In [16]:
p = 0.6
_, label1, label2 = info_dataset(stand_data_list,False)

###### Split the data set into train set and test set

In [17]:
train_set, test_set = [], []
max_label1, max_label2 = int(p * label1), int(p * label2)
total_label1, total_label2 = 0, 0
for sample in stand_data_list:
    if (total_label1 + total_label2) < (max_label1 + max_label2):
        train_set.append(sample)
        if sample[-1] == 1 and total_label1 < max_label1:
            total_label1 += 1
        else:
            total_label2 += 1
    else:
        test_set.append(sample)

In [18]:
print(len(train_set),len(test_set))

183 123


##### Define function to calculate the manhattan distance between two points

manhattan Distance - Wikipedia

In [19]:
def manhattan_dist(p1, p2): 
    if len(p1)==len(p2):
        r = 0 
        for i in range(0, len(p1)-1):
            r+=abs(p1[i]-p2[i])
        return r

##### Calculates the distance between a given sample and every other in the train set

Feeds its distances to a dictionary, the sort it and gets the nearest K neighbors; Then it counts witch of the labels is the most recurring, and returns it.

In [20]:
def knn(train_set, new_sample, K):
    dists, train_size = {}, len(train_set)
    
    for i in range(train_size):
        d = manhattan_dist(train_set[i], new_sample)
        dists[i] = d
    
    k_neighbors = sorted(dists, key=dists.get)[:K]
    
    qty_label1, qty_label2 = 0, 0
    for index in k_neighbors:
        if train_set[index][-1] == 1:
            qty_label1 += 1
        else:
            qty_label2 += 1
            
    if qty_label1 > qty_label2:
        return 1
    else:
        return 2

##### Example

In [21]:
print(test_set[0])
print(knn(train_set, test_set[0], 3))

[0.23533992449133045, -1.493486247310241, -0.5599913524831942, 1.0]
1


##### Counts the correct predictions of the test set with a given K

In [22]:
correct, K = 0, 3

for sample in test_set:
    label = knn(train_set, sample, K)
    if sample[-1] == label:
        correct += 1

In [23]:
# predicted value list
predict = []

for sample in test_set:
    label = knn(train_set,sample,K)
    predict.append(label)

In [24]:
# calulatae and show precision and recall

TP_FP = 0
TP_FN = 0
TP = 0

for i in range(len(predict)):
    if (predict[i] == 1) & (test_set[i][-1] == 1):
            TP += 1

for j in range(len(predict)):
    if predict[j] == 1:
        TP_FP +=1

for i in range(len(predict)):
    if test_set[i][-1] == 1:
        TP_FN +=1

In [25]:
print("Train set size: %d" % len(train_set))
print("Test set size: %d" % len(test_set))
print("Correct predicitons: %d" % correct)
print("True Positive: %d" %TP)
print("True Positive + False Positive: %d"%TP_FP)
print("True Positive + False Negative: %d"%TP_FN)
print("Accuracy: %.2f%%" % (100 * correct / len(test_set)))
print("Precision: %.2f%%"%(100*TP/TP_FP))
print("Recall: %.2f%%"% (100* TP/TP_FN))

Train set size: 183
Test set size: 123
Correct predicitons: 90
True Positive: 76
True Positive + False Positive: 92
True Positive + False Negative: 93
Accuracy: 73.17%
Precision: 82.61%
Recall: 81.72%
